[Drippypale](https://github.com/drippypale/ROSALIND)  
Email: drippypale@gmail.com  

Problem: **[Implement SmallParsimony](https://rosalind.info/problems/ba7f/)**

In [265]:
nodes = {} # our tree
seq_list = []

In [266]:
def haming_distance(s1, s2):
    return sum([s1[i] != s2[i] for i in range(len(s1))])

In [267]:
class Node:
    def __init__(self, value):
        self.value = value if not value.isnumeric() else ''
        self.children = list()
        self.parent = None
        self.is_leaf = False
        self.s = dict()
        self.tag = 0
        self.back_track = dict()

    def is_ripe(self):
        return self.children[0].tag == self.children[1].tag == 1 and self.tag == 0
    
    def print(self):
        res = list()
        if self.parent is not None:
            res.append(f'{self.value}->{self.parent.value}:{haming_distance(self.value, self.parent.value)}')
        for c in self.children:
            res.append(f'{self.value}->{c.value}:{haming_distance(self.value, c.value)}')
            res.extend(c.print())
        return res



In [268]:
with open('ba7f.in', 'r') as f:
    lines = f.read().splitlines()
    n = int(lines[0])
    for l in lines[1:]:
        n1, n2 = l.split('->')
        nodes[n1] = nodes.get(n1, Node(value=n1))
        nodes[n2] = nodes.get(n2, Node(value=n2))

        nodes[n1].children.append(nodes[n2])
        nodes[n2].parent = nodes[n1]
    
        if not n2.isnumeric():
            seq_list.append(n2)
            nodes[n2].is_leaf = True

In [269]:
root = max([_ for _ in nodes.items() if _[0].isnumeric()])[1]

In [270]:
m = len(seq_list[0])

In [271]:
def back_track(node: Node, k):
    node.value += k
    for i, c in enumerate(node.children):
        if not c.is_leaf:
            back_track(c, node.back_track[k][i])

In [272]:
internal_nodes = [node for node in nodes.values() if not node.is_leaf]

In [273]:
import math

get_ripe_nodes = lambda: [node for node in internal_nodes if node.is_ripe()]

total_parsimony_score = 0

for i in range(m):
    for node in nodes.values():
        node.tag = 1 if node.is_leaf else 0
        if node.is_leaf:
            node.s = {k: 0 if node.value[i] == k else math.inf for k in 'ACTG'}

    while len(get_ripe_nodes()) > 0:
        v: Node = get_ripe_nodes()[0]
        v.tag = 1
        for k in 'ACTG':
            left_min = min([(v.children[0].s[_] + int(_ != k), _) for _ in 'ACTG'], key=lambda x: x[0])
            right_min = min([(v.children[1].s[_] + int(_ != k), _) for _ in 'ACTG'], key=lambda x: x[0])
            v.s[k] = left_min[0] + right_min[0]
            v.back_track[k] = (left_min[1], right_min[1])
    
    min_ = min(list(root.s.items()), key=lambda x: x[1])

    total_parsimony_score += min_[1]
    back_track(root, min_[0])
          

In [274]:
with open('ba7f.out', 'w') as f:
    f.write(str(total_parsimony_score) + '\n')
    for l in root.print():
        f.write(l + '\n')